# Binary Classification using ANN in PyTorch

In [3]:

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

In [4]:
torch.__version__

'2.2.2'

In [5]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [6]:
# Load the dataset (assuming the same CSV file is available)
data = pd.read_csv("Social_Network_Ads.csv")

In [7]:
# Exploratory info
print(data.info())
print(data.Purchased.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Age              400 non-null    int64
 1   EstimatedSalary  400 non-null    int64
 2   Purchased        400 non-null    int64
dtypes: int64(3)
memory usage: 9.5 KB
None
Purchased
0    257
1    143
Name: count, dtype: int64


In [8]:
# Preprocessing
# Selecting relevant features and label
X = data[['Age', 'EstimatedSalary']].values  # Features
y = data['Purchased'].values  # Target

In [9]:
# Feature Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Reshape to (n_samples, 1)

In [11]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#to(device) ensures everything (model and data) runs on the same hardware,
#maximizing performance and avoiding errors.

In [12]:
# Send data to device
X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)

In [13]:
# Define the ANN model
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(2, 8),  # Input layer -> Hidden layer  #Linear(input,output) ---> similar to Dense
            nn.ReLU(),
            nn.Linear(8, 4),  # Hidden layer
            nn.ReLU(),
            nn.Linear(4, 1),  # Output layer
            nn.Sigmoid()      # Sigmoid for binary classification
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
# Instantiate the model
model = ANNModel().to(device)

In [ ]:
# Loss and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
n_epochs = 5
for epoch in range(n_epochs):
    model.train()  # Set model to training mode
    optimizer.zero_grad()  # Clear previous gradients
    outputs = model(X_train)  # Forward pass on training data
    loss = criterion(outputs, y_train)  # Compute loss on training predictions
    loss.backward()  # Backpropagation to compute gradients
    optimizer.step()  # Update weights using optimizer

    # Compute training accuracy
    predicted_train = (outputs > 0.5).float()
    train_acc = accuracy_score(y_train.cpu(), predicted_train.cpu())

    # Validation loss and accuracy
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, y_test)
        predicted_val = (val_outputs > 0.5).float()
        val_acc = accuracy_score(y_test.cpu(), predicted_val.cpu())

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")


In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predicted_classes = (predictions > 0.5).float()
    acc = accuracy_score(y_test.cpu(), predicted_classes.cpu())
    cm = confusion_matrix(y_test.cpu(), predicted_classes.cpu())

In [ ]:
print("\nTest Accuracy:", acc)
print("Confusion Matrix:\n", cm)